### Schrödingergleichung

$$i\hbar\frac{\partial \psi(x,t)}{\partial t} = \hat{H} \psi(x,t)$$

Für zeitunabhängige Hamiltonoperatoren ist die Zeitentwicklung eines Zustands

$$\psi(x,t) = \sum_{n=0}^{\infty} c_n e^{-iE_n(t-t_0)/\hbar } \,\phi^{(n)}(x), \quad c_n = \langle \phi^{(n)}|\psi(t_0)\rangle$$

mit orthogonalen Eigenzuständen $\phi^{(n)}$ und reellen Energieeigenwerten $E_n$ als Lösung der stationären Schrödingergleichung

$$\hat{H}\phi = E\phi$$

### Eigenzustände - numerisch

Räumlich beschränktes Teilchen in 1D:

$$\hat{H} = -\frac{\hbar^2}{2m}\frac{\mathrm{d}^2}{\mathrm{d}x^2} + V(x)$$

$$\phi(0) = \phi(L) = 0$$

$$\langle\phi|\psi\rangle = \int_0^L \phi^*(x) \psi(x) \mathrm{d}x$$

- Dimensionslose Größen ($x' = x / L, t' = \frac{t}{mL^2}, V' = mL^2V, E' = mL^2E$, wobei im Folgenden $'$ weggelassen) und Konvention $\hbar=1$:

$$\left[-\frac{1}{2}\frac{\mathrm{d}^2}{\mathrm{d}{x}^2} + V(x)\right]\mathclose{}\phi(x) = E\phi(x)$$

$$\phi(0) = \phi(1) = 0$$

- Diskretisierung ($\phi_j \approx \frac{\phi_{j-1}-2\phi_j+\phi_{j+1}}{\Delta x^2}$ mit $\phi_j = \phi(j\Delta x);\: \phi_0 = \phi_N = 0;\: \Delta x = 1/N$ ) ergibt ein lineares System mit tridiagonaler Matrix:

$$
\begin{bmatrix}\frac{1}{\Delta x^2}+V_1 & -\frac{1}{2 \Delta x^2} & \hphantom{\frac{1}{\Delta x^2}+V_1} &\\[10pt]
-\frac{1}{2 \Delta x^2} & \ddots & \ddots &\\[10pt]
& \ddots & \ddots & -\frac{1}{2 \Delta x^2}\\[10pt]
& \hphantom{\frac{1}{\Delta x^2}+V_1} & -\frac{1}{2 \Delta x^2} & \frac{1}{\Delta x^2}+V_{N-1}\end{bmatrix}
\begin{bmatrix} \phi_1 \\ \vdots \\ \phi_{N-1} \end{bmatrix} = E \begin{bmatrix} \phi_1 \\ \vdots \\ \phi_{N-1} \end{bmatrix}
$$

### Implementation

In [275]:
#%pip install plotly
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh_tridiagonal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML

def eigenstates(potential, N=500):
    """
    Lösung für ein räumlich beschränktes Teilchen.

    Parameters:
    -----------
    potential : callable
        Potential V(x) innerhalb eines unendlichen Potentialkasten für x in [0,1].
    N : int, optional (default=500)
        Anzahl der Diskretisierungspunkte.
    
    Returns:
    --------
    x : numpy.ndarray, shape (N,)
        Positionen.
    E_n : numpy.ndarray, shape (N-2,)
        Energieeigenwerte.
    phi_n : numpy.ndarray, shape (N-2, N)
        Eigenzustände phi_n(x) in Zeilen.
    """
    N -= 1
    dx = 1 / N
    x = np.linspace(0, 1, N+1)
    d = 1/dx**2 + np.vectorize(potential)(x[1:-1]) # diagonal
    e = np.full(len(d)-1, -1/(2*dx**2)) # off-diagonal
    E_n, phi_n = eigh_tridiagonal(d, e)
    phi_n = np.pad(phi_n.T, ((0, 0), (1, 1))) # include boundary values
    norm = np.sum(np.abs(phi_n)**2, axis=1) * dx # normalize rows
    phi_n /= norm[:, np.newaxis]
    return x, E_n, phi_n

def eigenstates_animation(x, potential, E_n, phi_n, n_max=10, fill_borders=True):
    # TODO: phi_n complex? -> plot Re, Im, abs + limit ylims different
    E_n = E_n[:n_max+1]
    N = len(E_n)
    phi_n = phi_n[:n_max+1,:]
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # fig = go.Figure()

    # Potential plot
    V = np.vectorize(potential)(x)
    fig.add_trace(
        go.Scatter(
            visible=True,
            name=r'$\text{Potential}$',
            x=x,
            y=V,
            line=dict(color='rgb(89, 117, 109)'),
            ),
        secondary_y=True)
    
    # Potential well border plot: arrows, optionally filled box
    arrow_length = max(1, np.max(V))
    arrows = []
    for arrow_x in [0, 1]:
        arrow = go.layout.Annotation(
            x=arrow_x, y=arrow_length,
            xref="x", yref="y2",
            ax=arrow_x, ay=0,
            axref="x", ayref='y2',
            text="", yanchor="top",
            showarrow=True,
            arrowhead=2, arrowwidth=2, arrowcolor='rgb(89, 117, 109)')
        arrows.append(arrow)
    fig.update_layout(annotations=arrows)
    if fill_borders:
        for box_x in [(-0.02, 0), (1, 1.02)]:
            fig.add_shape(
                type="rect",
                x0=box_x[0],
                y0=min(-1, np.min(V)),
                x1=box_x[1],
                y1=max(1, np.max(V)),
                fillcolor='rgb(145, 164, 163)',
                line=dict(width=2, color='rgb(145, 164, 163)'),
                xref="x", yref="y2")
     
    # Add traces and steps for slider
    steps = []
    for i, E_i in enumerate(E_n):
        # Add animated plot corresponding to slider
        visible_beginning = i == 0 # first one visible even before startin animation
        fig.add_trace(
            go.Scatter(
                visible=visible_beginning,
                name=rf'$\text{{Wellenfunktion n={i}}}$', # TEXT TODO {"yaxis":dict(title_text=f'Wellenfunktion $\phi_{{n={i}}}(x)', secondary_y=False)}
                x=x,
                y=phi_n[i,:],
                line=dict(color='rgb(245, 110, 73)')))

        # Add color alpha trail plot of all Eigenstates, TODO: only plot j<i
        for j, phi in enumerate(phi_n):
            alpha = 0.6 * np.exp(-0.8 * abs(j - i)) * (j<i)
            fig.add_trace(
                go.Scatter(
                    visible=False,
                    showlegend=False,
                    x=x,
                    y=phi,
                    line=dict(color=f'rgba(245, 110, 73, {alpha})')))

        # Visibility of traces depending depending on slider position:
        # True at slider position + trail (together length N+1) + first trace is always visible potential plot
        visible = [True] + [False]*(N+1)*i + [True]*(N+1) + [False]*(N+1)*(N-i-1)
        step = dict(
            method="update",
            args=[{"visible": visible}, {"title": rf"$\text{{Eigenzustand zu }}E_{{{i}}} = {E_i:.2f}\text{{ TODO:Einheiten.}}$"}],
            label=str(i))
        steps.append(step)

    # Create slider
    slider = go.layout.Slider(
        currentvalue={"prefix": "n: "},
        pad={"t": 50},
        steps=steps,
        len=1)

    fig.update_layout(
        sliders=[slider],
        title="Eigenzustände für gegebenes Potential.",
        showlegend=True)
    
    #fig.update_yaxes(title_text="Wellenfunktion", range=[np.min(phi_n)*1.05, np.max(phi_n)*1.05], secondary_y=False)
    #fig.update_yaxes(title_text='Potential', range=[min(-1, np.min(V))*1.05, arrow_length*1.05], secondary_y=True)
    fig.update_yaxes(title_text=r'$\phi$', showgrid=False, title_font=dict(color='rgb(245, 110, 73)'), tickfont=dict(color='rgb(245, 110, 73)'), secondary_y=False)
    fig.update_yaxes(title_text=r'$V$', showgrid=False, title_font=dict(color='rgb(89, 117, 109)'), tickfont=dict(color='rgb(89, 117, 109)'), secondary_y=True)
    fig.update_xaxes(title_text='$x$')

    return fig

In [277]:
def potential(x, mu=0.7, sigma=0.05):
    return -1e4 * np.exp(-(x - mu)**2 / (2*sigma**2))

""" def potential(x):
    return 0 """

x, E_n, phi_n = eigenstates(potential)

# potential plot matplotlib
""" fig = plt.figure()
plt.arrow(0, 0, 0, 1, color='tab:blue', length_includes_head=True, head_width=0.03, head_length=0.03)
plt.arrow(1, 0, 0, 1, color='tab:blue', length_includes_head=True, head_width=0.03, head_length=0.03)
plt.plot(x, np.vectorize(potential)(x)) """

fig = eigenstates_animation(x, potential, E_n, phi_n, fill_borders=False)
fig.show() # Use HTML(fig.to_html()) in jupyterlab instead.
fig.write_html('plot.html', include_mathjax='cdn')

### TODO

- Einheiten, E spacing plot
- more potentials: step, delta, ... add selection menu -> html OR interactive input field -> jhub
- plot analytical solutions; check normalization
- embed plot into HTML version of notebook (hide a view code cells before html export) OR embed only HTML plot in moodle etc..